In [ ]:
# upload files listed below from the folder (or you can simply upload all files)
# The `files` tab will be like
# - sample_data (this one is default in colab)
# - utils.py
# - policy.py
# - evaluate.py
# - pytorch_utils.py
# - expert_data.pkl
# - expert_policy.pkl

# !NOTE!: You need to copy your implementation of bc, dagger and policy_gradient 
# in the notebook here back to the python script when you submiting your code

In [ ]:
!apt-get install -y \
   libgl1-mesa-dev \
   libgl1-mesa-glx \
   libglew-dev \
   libosmesa6-dev \
   software-properties-common

!apt-get install -y patchelf 

!pip install free-mujoco-py
!pip install gym==0.22.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libglew-dev is already the newest version (2.1.0-4).
libgl1-mesa-dev is already the newest version (21.2.6-0ubuntu0.1~20.04.2).
libgl1-mesa-glx is already the newest version (21.2.6-0ubuntu0.1~20.04.2).
libosmesa6-dev is already the newest version (21.2.6-0ubuntu0.1~20.04.2).
software-properties-common is already the newest version (0.99.9.11).
0 upgraded, 0 newly installed, 0 to remove and 27 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
patchelf is already the newest version (0.10-2build1).
0 upgraded, 0 newly installed, 0 to remove and 27 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Include this at the top of your colab code
import os
if not os.path.exists('.mujoco_setup_complete'):
  # Get the prereqs
  !apt-get -qq update
  !apt-get -qq install -y libosmesa6-dev libgl1-mesa-glx libglfw3 libgl1-mesa-dev libglew-dev patchelf
  # Get Mujoco
  !mkdir ~/.mujoco
  !wget -q https://mujoco.org/download/mujoco210-linux-x86_64.tar.gz -O mujoco.tar.gz
  !tar -zxf mujoco.tar.gz -C "$HOME/.mujoco"
  !rm mujoco.tar.gz
  # Add it to the actively loaded path and the bashrc path (these only do so much)
  !echo 'export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$HOME/.mujoco/mujoco210/bin' >> ~/.bashrc 
  !echo 'export LD_PRELOAD=$LD_PRELOAD:/usr/lib/x86_64-linux-gnu/libGLEW.so' >> ~/.bashrc 
  # THE ANNOYING ONE, FORCE IT INTO LDCONFIG SO WE ACTUALLY GET ACCESS TO IT THIS SESSION
  !echo "/root/.mujoco/mujoco210/bin" > /etc/ld.so.conf.d/mujoco_ld_lib_path.conf
  !ldconfig
  # Install Mujoco-py
  !pip3 install -U 'mujoco-py<2.2,>=2.1'
  # run once
  !touch .mujoco_setup_complete

try:
  if _mujoco_run_once:
    pass
except NameError:
  _mujoco_run_once = False
if not _mujoco_run_once:
  # Add it to the actively loaded path and the bashrc path (these only do so much)
  try:
    os.environ['LD_LIBRARY_PATH']=os.environ['LD_LIBRARY_PATH'] + ':/root/.mujoco/mujoco210/bin'
  except KeyError:
    os.environ['LD_LIBRARY_PATH']='/root/.mujoco/mujoco210/bin'
  try:
    os.environ['LD_PRELOAD']=os.environ['LD_PRELOAD'] + ':/usr/lib/x86_64-linux-gnu/libGLEW.so'
  except KeyError:
    os.environ['LD_PRELOAD']='/usr/lib/x86_64-linux-gnu/libGLEW.so'
  # presetup so we don't see output on first env initialization
  import mujoco_py
  _mujoco_run_once = True

In [ ]:
# Test whether mujoco and gym is correctly installed
# You are expected to get an output of `(11, )`
import gym
import mujoco_py
env = gym.make("Reacher-v2")
print(env.reset().shape)

(11,)


In [ ]:
import torch
import torch.optim as optim
import numpy as np
from utils import log_density, rollout
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# NOTE: copy your implementation back to bc.py when you submit the code
def simulate_policy_bc(env, policy, expert_data, num_epochs=500, episode_length=50, 
                       batch_size=32):
    
    # Hint: Just flatten your expert dataset and use standard pytorch supervised learning code to train the policy. 
    optimizer = optim.Adam(list(policy.parameters()))
    idxs = np.array(range(len(expert_data)))
    num_batches = len(idxs)*episode_length // batch_size
    losses = []
    for epoch in range(num_epochs): 
        ## TODO Students
        np.random.shuffle(idxs)
        running_loss = 0.0
        for i in range(num_batches):
            optimizer.zero_grad()
            # TODO start: Fill in your behavior cloning implementation here
            
            # TODO end
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        if epoch % 50 == 0:
            print('[%d] loss: %.8f' %
                (epoch, running_loss / 10.))
        losses.append(loss.item())

In [ ]:
# NOTE: copy your implementation back to dagger.py when you submit the code
def simulate_policy_dagger(env, policy, expert_paths, expert_policy=None, num_epochs=500, episode_length=50,
                            batch_size=32, num_dagger_iters=10, num_trajs_per_dagger=10):
    
    # TODO: Fill in your dagger implementation here. Make sure to plot your intermediate returns
    
    # Hint: Loop through num_dagger_iters iterations, at each iteration train a policy on the current dataset.
    # Then rollout the policy, use relabel_action to relabel the actions along the trajectory with "expert_policy" and then add this to current dataset
    # Repeat this so the dataset grows with states drawn from the policy, and relabeled actions using the expert.
    # Optimizer code
    optimizer = optim.Adam(list(policy.parameters()))
    losses = []
    returns = []

    trajs = expert_paths
    # Dagger iterations
    for dagger_itr in range(num_dagger_iters):
        idxs = np.array(range(len(trajs)))
        num_batches = len(idxs)*episode_length // batch_size
        losses = []
        # Train the model with Adam
        for epoch in range(num_epochs):
            running_loss = 0.0
            for i in range(num_batches):
                optimizer.zero_grad()
                # TODO start: Fill in your behavior cloning implementation here
                
                # TODO end
                loss.backward()
                optimizer.step()

                # print statistics
                running_loss += loss.item()
            # print('[%d, %5d] loss: %.8f' %(epoch + 1, i + 1, running_loss))
            losses.append(loss.item())

        # Collecting more data for dagger
        trajs_recent = []
        for k in range(num_trajs_per_dagger):
            env.reset()
            # TODO start: Rollout the policy on the environment to collect more data, relabel them, add them into trajs_recent
            
            # TODO end

        trajs += trajs_recent
        mean_return = np.mean(np.array([traj['rewards'].sum() for traj in trajs_recent]))
        print("Average DAgger return is " + str(mean_return))
        returns.append(mean_return)

In [ ]:
# NOTE: copy your implementation back to policy_gradient.py when you submit the code
def train_model(policy, baseline, trajs, policy_optim, baseline_optim, gamma=0.99, baseline_train_batch_size=64, baseline_num_epochs=5):
    # Fill in your policy gradient implementation here

    # TODO: Compute the returns on the current batch of trajectories
    # Hint: Go through all the trajectories in trajs and compute their return to go: discounted sum of rewards from that timestep to the end. 
    # Hint: This is easy to do if you go backwards in time and sum up the reward as a running sum. 
    # Hint: Remember that return to go is return = r[t] + gamma*r[t+1] + gamma^2*r[t+2] + ...
    states_all = []
    actions_all = []
    returns_all = []
    for traj in trajs:
        states_singletraj = traj['observations']
        actions_singletraj = traj['actions']
        rewards_singletraj = traj['rewards']
        returns_singletraj = np.zeros_like(rewards_singletraj)
        # TODO start
            
        # TODO end
        states_all.append(states_singletraj)
        actions_all.append(actions_singletraj)
        returns_all.append(returns_singletraj)
    states = np.concatenate(states_all)
    actions = np.concatenate(actions_all)
    returns = np.concatenate(returns_all)
    
    # TODO: Normalize the returns by subtracting mean and dividing by std
    # Hint: Just do return - return.mean()/ (return.std() + EPS), where EPS is a small constant for numerics
    # TODO start
    
    # TODO end
    
    # TODO: Train baseline by regressing onto returns
    # Hint: Regress the baseline from each state onto the above computed return to go. You can use similar code to behavior cloning to do so. 
    # Hint: Iterate for baseline_num_epochs with batch size = baseline_train_batch_size
    criterion = torch.nn.MSELoss()
    n = len(states)
    arr = np.arange(n)
    for epoch in range(baseline_num_epochs):
        np.random.shuffle(arr)
        for i in range(n // baseline_train_batch_size):
            batch_index = arr[baseline_train_batch_size * i: baseline_train_batch_size * (i + 1)]
            batch_index = torch.LongTensor(batch_index).to(device)
            # TODO start         
            
            # TODO end
            baseline_optim.zero_grad()
            loss.backward()
            baseline_optim.step()
            
    # TODO: Train policy by optimizing surrogate objective: -log prob * (return - baseline)
    # Hint: Policy gradient is given by: \grad log prob(a|s)* (return - baseline)
    # Hint: Return is computed above, you can computer log_probs using the log_density function imported. 
    # Hint: You can predict what the baseline outputs for every state.  
    # Hint: Then simply compute the surrogate objective by taking the objective as -log prob * (return - baseline)
    # Hint: You can then use standard pytorch machinery to take *one* gradient step on the policy
    mu, std, logstd = policy(torch.Tensor(states).to(device))
    log_policy = log_density(torch.Tensor(actions).to(device), mu, std, logstd)
    baseline_pred = baseline(torch.from_numpy(states).float().to(device))
    # TODO start
    
    # TODO end
    
    policy_optim.zero_grad()
    loss.backward()
    policy_optim.step()
    
    del states, actions, returns, states_all, actions_all, returns_all

# Training loop for policy gradient
def simulate_policy_pg(env, policy, baseline, num_epochs=20000, max_path_length=200, pg_batch_size=100, 
                        gamma=0.99, baseline_train_batch_size=64, baseline_num_epochs=5, print_freq=10, render=False):
    policy_optim = optim.Adam(policy.parameters())
    baseline_optim = optim.Adam(baseline.parameters())

    for iter_num in range(num_epochs):
        sample_trajs = []

        # Sampling trajectories
        for it in range(pg_batch_size):
            sample_traj = rollout(env=env, agent=policy, episode_length=max_path_length, agent_name='pg', render=render)
            sample_trajs.append(sample_traj)
        
        # Logging returns occasionally
        if iter_num % print_freq == 0:
            rewards_np = np.mean(np.asarray([traj['rewards'].sum() for traj in sample_trajs]))
            path_length = np.max(np.asarray([traj['rewards'].shape[0] for traj in sample_trajs]))
            print("Episode: {}, reward: {}, max path length: {}".format(iter_num, rewards_np, path_length))

        # Training model
        train_model(policy, baseline, sample_trajs, policy_optim, baseline_optim, gamma=gamma, 
                    baseline_train_batch_size=baseline_train_batch_size, baseline_num_epochs=baseline_num_epochs)


In [ ]:

class Args:
    def __init__(self, task, test, render=False):
        self.task = task # 'behavior_cloning', 'dagger' or 'policy_gradient'
        self.test = test # whether test only
        self.render = render # whether to render during test
args = Args('policy_gradient', False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import gym
import numpy as np
import time
import matplotlib.pyplot as plt
import torch
import argparse
import torch.nn as nn

from utils import BCPolicy, generate_paths, get_expert_data, RLPolicy, RLBaseline
from policy import MakeDeterministic
import pytorch_utils as ptu
from evaluate import evaluate

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('using device', device)

torch.manual_seed(0)
import random
random.seed(0)
np.random.seed(0)

if __name__ == '__main__':

    if args.task == 'policy_gradient':
        # Define environment
        env = gym.make("InvertedPendulum-v2")

        # Define policy and value function
        hidden_dim_pol = 64
        hidden_depth_pol = 2
        hidden_dim_baseline = 64
        hidden_depth_baseline = 2
        policy = RLPolicy(env.observation_space.shape[0], env.action_space.shape[0], hidden_dim=hidden_dim_pol, hidden_depth=hidden_depth_pol)
        baseline = RLBaseline(env.observation_space.shape[0], hidden_dim=hidden_dim_baseline, hidden_depth=hidden_depth_baseline)
        policy.to(device)
        baseline.to(device)

        # Training hyperparameters
        num_epochs=100
        max_path_length=200
        pg_batch_size=100
        gamma=0.99
        baseline_train_batch_size=64
        baseline_num_epochs=5
        print_freq=10

        if not args.test:
            # Train policy gradient
            simulate_policy_pg(env, policy, baseline, num_epochs=num_epochs, max_path_length=max_path_length, pg_batch_size=pg_batch_size, 
                            gamma=gamma, baseline_train_batch_size=baseline_train_batch_size, baseline_num_epochs=baseline_num_epochs, print_freq=print_freq, render=False)
            torch.save(policy.state_dict(), 'pg_final.pth')
        else:
            policy.load_state_dict(torch.load(f'pg_final.pth'))
        evaluate(env, policy, 'pg', num_validation_runs=100, episode_length=max_path_length, render=args.render)

    if args.task == 'behavior_cloning':
        # Define environment
        env = gym.make("Reacher-v2")

        # Define policy
        hidden_dim = 128
        hidden_depth = 2
        obs_size = env.observation_space.shape[0]
        ac_size = env.action_space.shape[0]
        print(obs_size, ac_size, hidden_dim, hidden_depth)
        policy = BCPolicy(obs_size, ac_size, hidden_dim=hidden_dim, hidden_depth=hidden_depth) # 10 dimensional latent
        policy.to(device)

        # Get the expert data
        file_path = 'expert_data.pkl'
        expert_data = get_expert_data(file_path)
        # print(expert_data)
        # Training hyperparameters
        episode_length = 50
        num_epochs = 500
        batch_size = 32
        num_validation_runs = 100

        if not args.test:
        # Train behavior cloning
            simulate_policy_bc(env, policy, expert_data, num_epochs=num_epochs, episode_length=episode_length,
                            batch_size=batch_size)
            torch.save(policy.state_dict(), 'bc_final.pth')
        else:
            policy.load_state_dict(torch.load(f'bc_final.pth'))
        evaluate(env, policy, 'bc', num_validation_runs=num_validation_runs, episode_length=episode_length, render=args.render)

    if args.task == 'dagger':
        # Define environment
        env = gym.make("Reacher-v2")

        # Policy
        hidden_dim = 1000
        hidden_depth = 3
        obs_size = env.observation_space.shape[0]
        ac_size = env.action_space.shape[0]
        policy = BCPolicy(obs_size, ac_size, hidden_dim=hidden_dim, hidden_depth=hidden_depth) # 10 dimensional latent
        policy.to(device)

        # Get the expert data
        file_path = 'expert_data.pkl'
        expert_data = get_expert_data(file_path)

        # Load interactive expert
        expert_policy = torch.load('expert_policy.pkl', map_location=torch.device(device))
        print("Expert policy loaded")
        expert_policy.to(device)
        ptu.set_gpu_mode(True)

        # Training hyperparameters
        episode_length = 50
        num_epochs = 400
        batch_size = 32
        num_dagger_iters = 10
        num_trajs_per_dagger = 10
        num_validation_runs = 100

        if not args.test:
        # Train DAgger
            simulate_policy_dagger(env, policy, expert_data, expert_policy, num_epochs=num_epochs, episode_length=episode_length,
                               batch_size=batch_size, num_dagger_iters=num_dagger_iters, num_trajs_per_dagger=num_trajs_per_dagger)
            torch.save(policy.state_dict(), 'dagger_final.pth')
        else:
            policy.load_state_dict(torch.load(f'dagger_final.pth'))
        evaluate(env, policy, 'dagger', num_validation_runs=num_validation_runs, episode_length=episode_length, render=args.render)